In [43]:
import numpy as np
import math
t = 0 #time
Th = [] #collector of theta
num_theta = 0 #num theta
A = [] #collector of arm
num_arms = 0 #number of arms
K = 3 #dimension of arm x, y, theta
Y = [] #collector of arm y
num_arms_y = 0 #number of arms of player two
M = [] #world
num_M = 0
M_vero = 0
theta_vero = 0
N = [] #best and second best x
B = [] #best x

delta = 0.05
sigma = 0.1 #noise

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1


class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        
class arm():
    arm_vector = np.array([])
    id = 0
    worst_y = -1
    
    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A, N
        A.append(self)
        self.id = num_arms
        num_arms +=1
        N.append(0)
        B.append(0)
                            

    def pull_arm(self, y_index):
        reward = np.dot(self.arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[y_index].arm_vector))) + np.random.normal(0, +sigma)
        Y[y_index].pull_arm(reward, self.id)
        global N
        N[self.id] += 1
        return reward
    
    def find_worst_y(self):
        print("find worst y")
        mini = 100000000
        index = - 1
        for i in range(0, num_arms_y):
            curr = Y[i].lower_bound(self.arm_vector)
            if curr < mini:
                index = i
                mini = curr
        self.worst_y = index
        #print("mini", mini)
        return mini
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

class arm_y():
    arm_vector = np.array([])
    id = 0
    best_theta = -1
    R = [] #collection of reward
    A_id = [] #id of the i^th arm played
    times = 0
    theta_stimato = [0.0, 0.0, 0.0]

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y +=1
        
    def pull_arm(self, reward, id_arm):
        self.R.append(reward)
        self.A_id.append(id_arm)
        self.times += 1 
    
    def calcola_A(self):
        res = 0
        for i in range(0,self.times):
            #print("i", i)
            if i == 0:
                #print("i==0, print res", res)
                #print(" A[self.A_id[i]].arm_vector",  A[self.A_id[i]].arm_vector)
                res = np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
                #print("res", res)
            else:
                #print("i!=0, print i, res", i, res)
                res += np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
        #print("A", res)
        return res

    def calcola_b(self):
        res = []
        global A
        for i in range(0,self.times):
            curr = A[self.A_id[i]].arm_vector
            curr = [self.R[i] * xx for xx in curr]
            if i == 0:
                res = curr
            else:
                [x + y for x, y in zip(res, curr)]
        return res
        
    def stima_theta(self):
        if (self.times == 0):
            return self.theta_stimato
        theta_s = np.dot(np.linalg.inv(self.calcola_A()), self.calcola_b())
        self.theta_stimato = theta_s
        return theta_s
    
    def media_reward(self, arm_vector):
        return np.dot(arm_vector, np.transpose(self.stima_theta()))
    
    def calcola_bound(self, arm_vector):
        ris = 2*sigma*math.sqrt(2)*norm_2_mat(arm_vector, np.linalg.inv(self.calcola_A()))*math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta))
        #print("sqrt", math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta)))
        #print("bound", ris)
        return ris
    
    def upper_bound(self, arm_vector):
        return self.media_reward(arm_vector) + self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
    def lower_bound(self, arm_vector):
        return self.media_reward(arm_vector) - self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def run():
    global t, A, Y
    t += 1
    print("TIME", t)
    maxi = - 1000
    for i in range(0, num_arms):
        curr = Y[A[i].worst_y].media_reward(A[i].arm_vector)
        print("curr", curr)
        if curr > maxi:
            best = i
            maxi = curr
        print("i term and curr value for best, x", i, "y", A[i].worst_y, curr)
    print("best", best, A[best].worst_y)
    maxi = -1000
    for i in range(0, num_arms):
        if i is not best:
            curr = Y[A[i].worst_y].upper_bound(A[i].arm_vector)
            if curr > maxi:
                second_best = i
                maxi = curr
    print("second best", second_best, A[second_best].worst_y)
    B[best] += 1
    A[best].pull_arm(A[best].worst_y)
    A[second_best].pull_arm(A[second_best].worst_y)
        
        
        

In [44]:
a = arm([0.6, 1.1, 0.2])
b = arm([0.3, 0.1, 2.0])
c = arm([0.8, 1.1, 1.3])
m = world([[0.9, 2.5, 1.2], [1.3, 0.4, 2.5], [3.2, 2.1, 0.8]])
j = arm_y([1.0, 0.9, 1.2])
k = arm_y([1.4, 2.3, 0.5])
l = arm_y([2.7, 0.2, 3])
for i in range(0, num_arms):
    for j in range(0, num_arms_y):
        curr = np.dot(np.dot(A[i].arm_vector, m.matrix), Y[j].arm_vector)
        print(i, j, curr)

0 0 9.09
0 1 10.896999999999998
0 2 18.409000000000002
1 0 13.943
1 1 22.102
1 2 25.988000000000003
2 0 16.663
2 1 23.099999999999998
2 2 32.321


In [45]:
A[0].pull_arm(0)
A[0].pull_arm(1)
A[0].pull_arm(2)

A[1].pull_arm(0)
A[1].pull_arm(1)
A[1].pull_arm(2)

A[2].pull_arm(0)
A[2].pull_arm(1)
A[2].pull_arm(2)

A[0].pull_arm(0)
A[0].pull_arm(1)
A[0].pull_arm(2)

A[1].pull_arm(0)
A[1].pull_arm(1)
A[1].pull_arm(2)

A[2].pull_arm(0)
A[2].pull_arm(1)
A[2].pull_arm(2)

32.53591767488665

In [46]:
A[0].find_worst_y()
print("x", 0, "y", A[0].worst_y)
A[1].find_worst_y()
print("x", 1, "y", A[0].worst_y)
A[2].find_worst_y()
print("x", 2, "y", A[0].worst_y)

find worst y
x 0 y 0
find worst y
x 1 y 0
find worst y
x 2 y 0


In [47]:
res = np.dot(np.transpose(A[0].arm_vector), A[0].arm_vector)
res

1.6100000000000003

In [48]:
Y[0].calcola_bound(A[1].arm_vector)

0.4299874444141855

In [49]:
run()

TIME 1
curr 3.8398437500000004
i term and curr value for best, x 0 y 0 3.8398437500000004
curr 0.0
i term and curr value for best, x 1 y 0 0.0
curr 3.6660156250000004
i term and curr value for best, x 2 y 0 3.6660156250000004
best 0 0
second best 2 0


In [50]:
run()

TIME 2
curr 3.029888577036064
i term and curr value for best, x 0 y 0 3.029888577036064
curr -3.0020430585864233e-13
i term and curr value for best, x 1 y 0 -3.0020430585864233e-13
curr 7.167599846980011e-13
i term and curr value for best, x 2 y 0 7.167599846980011e-13
best 0 0
second best 2 0


In [51]:
print(Y[0].R)
print(Y[0].A_id)

[9.089665731109752, 11.024652940261637, 18.379373339722804, 13.862985059718168, 22.229560693360206, 25.951580287662015, 16.658552594772292, 23.081687403859217, 32.24791942030416, 9.107091684213623, 10.939752714847371, 18.35806518590698, 13.904274002199772, 22.0801794797183, 25.86330288069395, 16.60881491470531, 22.958908459760536, 32.53591767488665, 9.033705953124523, 16.74633193288594, 9.068964786262177, 16.691603698385503]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 2, 0, 2]


In [37]:
for i in range(0, 200):
    run()

TIME 3
curr 2.3313155274607125
i term and curr value for best, x 0 y 0 2.3313155274607125
curr -9.414691248821327e-14
i term and curr value for best, x 1 y 0 -9.414691248821327e-14
curr 2.149391775674303e-13
i term and curr value for best, x 2 y 0 2.149391775674303e-13
best 0 0
second best 2 0
TIME 4
curr 1.5542103516406032
i term and curr value for best, x 0 y 0 1.5542103516406032
curr -9.769962616701378e-15
i term and curr value for best, x 1 y 0 -9.769962616701378e-15
curr 3.2018832030189515e-13
i term and curr value for best, x 2 y 0 3.2018832030189515e-13
best 0 0
second best 2 0
TIME 5
curr 1.5542103516405694
i term and curr value for best, x 0 y 0 1.5542103516405694
curr 2.5757174171303632e-14
i term and curr value for best, x 1 y 0 2.5757174171303632e-14
curr 2.6201263381153694e-14
i term and curr value for best, x 2 y 0 2.6201263381153694e-14
best 0 0
second best 2 0
TIME 6
curr 1.5542103516404393
i term and curr value for best, x 0 y 0 1.5542103516404393
curr -2.1316282072803

curr 0.2664360602812964
i term and curr value for best, x 0 y 0 0.2664360602812964
curr 6.672440377997191e-14
i term and curr value for best, x 1 y 0 6.672440377997191e-14
curr -1.220135104063047e-13
i term and curr value for best, x 2 y 0 -1.220135104063047e-13
best 0 0
second best 1 0
TIME 37
curr 0.25903505860681947
i term and curr value for best, x 0 y 0 0.25903505860681947
curr 6.80566714095221e-14
i term and curr value for best, x 1 y 0 6.80566714095221e-14
curr -1.0313971898767704e-13
i term and curr value for best, x 2 y 0 -1.0313971898767704e-13
best 0 0
second best 1 0
TIME 38
curr 0.252034111076921
i term and curr value for best, x 0 y 0 0.252034111076921
curr 7.66053886991358e-14
i term and curr value for best, x 1 y 0 7.66053886991358e-14
curr -1.2323475573339238e-13
i term and curr value for best, x 2 y 0 -1.2323475573339238e-13
best 0 0
second best 1 0
TIME 39
curr 0.24540163446964172
i term and curr value for best, x 0 y 0 0.24540163446964172
curr 8.248957072964913e-14


curr 4.934941344458821e-14
i term and curr value for best, x 2 y 0 4.934941344458821e-14
best 0 0
second best 1 0
TIME 65
curr 0.14570722046628193
i term and curr value for best, x 0 y 0 0.14570722046628193
curr -2.8199664825478976e-14
i term and curr value for best, x 1 y 0 -2.8199664825478976e-14
curr 6.034062138837726e-14
i term and curr value for best, x 2 y 0 6.034062138837726e-14
best 0 0
second best 1 0
TIME 66
curr 0.14346557092063428
i term and curr value for best, x 0 y 0 0.14346557092063428
curr -3.7414515929867775e-14
i term and curr value for best, x 1 y 0 -3.7414515929867775e-14
curr 5.1736392947532295e-14
i term and curr value for best, x 2 y 0 5.1736392947532295e-14
best 0 0
second best 1 0
TIME 67
curr 0.14129185014911477
i term and curr value for best, x 0 y 0 0.14129185014911477
curr -3.441691376337985e-14
i term and curr value for best, x 1 y 0 -3.441691376337985e-14
curr 5.71209746169643e-14
i term and curr value for best, x 2 y 0 5.71209746169643e-14
best 0 0
seco

curr 0.10027163558963825
i term and curr value for best, x 0 y 0 0.10027163558963825
curr -8.099076964640517e-14
i term and curr value for best, x 1 y 0 -8.099076964640517e-14
curr 1.2090328738167955e-13
i term and curr value for best, x 2 y 0 1.2090328738167955e-13
best 0 0
second best 1 0
TIME 95
curr 0.09920491606208484
i term and curr value for best, x 0 y 0 0.09920491606208484
curr -8.54316617449058e-14
i term and curr value for best, x 1 y 0 -8.54316617449058e-14
curr 1.3231082895970303e-13
i term and curr value for best, x 2 y 0 1.3231082895970303e-13
best 0 0
second best 1 0
TIME 96
curr 0.0981606537877579
i term and curr value for best, x 0 y 0 0.0981606537877579
curr -7.549516567451064e-14
i term and curr value for best, x 1 y 0 -7.549516567451064e-14
curr 1.4241385848379196e-13
i term and curr value for best, x 2 y 0 1.4241385848379196e-13
best 0 0
second best 1 0
TIME 97
curr 0.09713814697745622
i term and curr value for best, x 0 y 0 0.09713814697745622
curr -7.81041897823

second best 1 0
TIME 125
curr 0.07520372669221012
i term and curr value for best, x 0 y 0 0.07520372669221012
curr -8.032463583163008e-14
i term and curr value for best, x 1 y 0 -8.032463583163008e-14
curr 1.4652168367490503e-13
i term and curr value for best, x 2 y 0 1.4652168367490503e-13
best 0 0
second best 1 0
TIME 126
curr 0.07460209687867227
i term and curr value for best, x 0 y 0 0.07460209687867227
curr -8.154588115871775e-14
i term and curr value for best, x 1 y 0 -8.154588115871775e-14
curr 1.4577228313328305e-13
i term and curr value for best, x 2 y 0 1.4577228313328305e-13
best 0 0
second best 1 0
TIME 127
curr 0.07401001674471236
i term and curr value for best, x 0 y 0 0.07401001674471236
curr -7.904787935331115e-14
i term and curr value for best, x 1 y 0 -7.904787935331115e-14
curr 1.3908318940991649e-13
i term and curr value for best, x 2 y 0 1.3908318940991649e-13
best 0 0
second best 2 0
TIME 128
curr 0.07342726070735595
i term and curr value for best, x 0 y 0 0.07342

curr 1.1296519275560968e-13
i term and curr value for best, x 2 y 0 1.1296519275560968e-13
best 0 0
second best 1 0
TIME 153
curr 0.06135040861733676
i term and curr value for best, x 0 y 0 0.06135040861733676
curr -5.867528685143952e-14
i term and curr value for best, x 1 y 0 -5.867528685143952e-14
curr 1.0860756738395594e-13
i term and curr value for best, x 2 y 0 1.0860756738395594e-13
best 0 0
second best 2 0
TIME 154
curr 0.06094942555447631
i term and curr value for best, x 0 y 0 0.06094942555447631
curr -5.806466418789569e-14
i term and curr value for best, x 1 y 0 -5.806466418789569e-14
curr 1.0608181000293371e-13
i term and curr value for best, x 2 y 0 1.0608181000293371e-13
best 0 0
second best 2 0
TIME 155
curr 0.06055365006386984
i term and curr value for best, x 0 y 0 0.06055365006386984
curr -5.5067062021407764e-14
i term and curr value for best, x 1 y 0 -5.5067062021407764e-14
curr 1.0977330155981235e-13
i term and curr value for best, x 2 y 0 1.0977330155981235e-13
best

curr -4.546363285840016e-14
i term and curr value for best, x 1 y 0 -4.546363285840016e-14
curr 8.439082765931971e-14
i term and curr value for best, x 2 y 0 8.439082765931971e-14
best 0 0
second best 2 0
TIME 181
curr 0.051807011721309255
i term and curr value for best, x 0 y 0 0.051807011721309255
curr -4.6629367034256575e-14
i term and curr value for best, x 1 y 0 -4.6629367034256575e-14
curr 7.911726829235022e-14
i term and curr value for best, x 2 y 0 7.911726829235022e-14
best 0 0
second best 2 0
TIME 182
curr 0.05152078513721475
i term and curr value for best, x 0 y 0 0.05152078513721475
curr -4.474198789239381e-14
i term and curr value for best, x 1 y 0 -4.474198789239381e-14
curr 8.534839501805891e-14
i term and curr value for best, x 2 y 0 8.534839501805891e-14
best 0 0
second best 2 0
TIME 183
curr 0.051237703900197995
i term and curr value for best, x 0 y 0 0.051237703900197995
curr -4.446443213623752e-14
i term and curr value for best, x 1 y 0 -4.446443213623752e-14
curr 8

In [38]:
print(N)
print(B)

[208, 109, 105]
[202, 0, 0]


In [39]:
for i in range(0, num_arms_y):
    curr = Y[i].stima_theta()
    print("stima",curr)
    print("vero", np.dot(m.matrix, Y[i].arm_vector))

stima [-0.68748816  0.40186506  0.08302997]
vero [4.59 4.66 6.05]
stima [-0.53125     2.859375   -0.28515625]
vero [7.61 3.99 9.71]
stima [-0.53125     2.859375   -0.28515625]
vero [ 6.53 11.09 11.46]


In [40]:
Y[0].R

[9.325262109843294,
 10.97094368604229,
 18.30555154071237,
 13.891994692265909,
 22.24298383018907,
 25.783850292225612,
 16.729360612734702,
 23.09393420449178,
 32.56247182680476,
 9.160453652529473,
 10.928909036015357,
 18.350947901793113,
 13.952406074137926,
 22.220024124200723,
 25.96714114042375,
 16.75555881889119,
 23.218788938328853,
 32.348054350124876,
 8.990726098804702,
 16.71956427339252,
 9.111191206889687,
 16.679478996362125,
 9.074818924465797,
 16.635814516070504,
 9.014222082218,
 16.571878670139036,
 9.20437766044283,
 16.631482900786896,
 8.980612791734629,
 16.775412404518416,
 9.147735527999592,
 16.701837155149143,
 9.0117496335234,
 13.804891708630231,
 9.04998097540249,
 13.894846405094341,
 9.048943426293416,
 13.98118126927599,
 9.083886297664149,
 13.971839079098224,
 9.254878376315228,
 13.946913553088585,
 8.999614779030718,
 13.904642325692766,
 9.040568229655067,
 14.113384088529793,
 9.161515126695909,
 13.877651163132569,
 9.034456690346781,
 14.0

In [41]:
A[0].find_worst_y()
print("x", 0, "y", A[0].worst_y)
A[1].find_worst_y()
print("x", 1, "y", A[0].worst_y)
A[2].find_worst_y()
print("x", 2, "y", A[0].worst_y)

find worst y
x 0 y 0
find worst y
x 1 y 0
find worst y
x 2 y 0
